In [ ]:
MESSAGES_HF_SHORT = [
    {"role": "system", "content": "You are a helpful assistant. Always tell the user before calling a function."},
    {"role": "user", "content": "What's the weather in Tokyo?"},
]

MESSAGES_HF_LONG = [
    {"role": "system", "content": "You are a helpful assistant."},
    # tool call 1: no preamble
    {"role": "user", "content": "QUESTION 1"},
    {
        "role": "assistant",
        "thinking": "REASONING 1",
        "content": "",
        "tool_calls": [
            {
                "type": "function",
                "function": {"name": "get_weather", "arguments": {"location": "Tokyo, JP", "unit": "celsius"}},
            }
        ],
    },
    {"role": "tool", "name": "get_weather", "content": "FUNCTION 1"},
    {"role": "assistant", "thinking": "REASONING 1.2", "content": "FINAL 1"},
    # tool call 2, with preamble
    {"role": "user", "content": "QUESTION 2"},
    {
        "role": "assistant",
        "thinking": "REASONING 2",
        # "content": "PREAMBLE 2",
        "tool_calls": [
            {
                "type": "function",
                "function": {"name": "get_weather", "arguments": {"location": "Tokyo, JP", "unit": "fahrenheit"}},
            }
        ],
    },
    {"role": "tool", "name": "get_weather", "content": "FUNCTION 2"},
    {"role": "assistant", "thinking": "REASONING 2.2", "content": "FINAL 2"},
]

In [ ]:
from transformers import AutoTokenizer


def get_weather(location: str):
    """
    Get the weather at a location.

    Args:
        location: The location
    """


tok = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")
print(
    tok.apply_chat_template(
        MESSAGES_HF_SHORT, tokenize=False, tools=[get_weather], add_generation_prompt=True, reasoning_effort="low"
    )
)

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("openai/gpt-oss-20b")
output = model.generate(
    tok.apply_chat_template(MESSAGES_HF_SHORT, tools=[get_weather], add_generation_prompt=True, reasoning_effort="low"),
    max_new_tokens=2048,
    temperature=1,
    top_p=1,
)
# decode to tokens
# the completion shouldn't include the prompt or stop token
content = tok.decode(output[0])
print(content)

In [ ]:
from openai_harmony import DeveloperContent, Message, Role, SystemContent

MESSAGES_HARMONY = [
    Message.from_role_and_content(Role.SYSTEM, SystemContent.new()),
    Message.from_role_and_content(
        Role.DEVELOPER, DeveloperContent.new().with_instructions("You are a helpful assistant.")
    ),
    # tool call 1: no preamble
    Message.from_role_and_content(Role.USER, "What's the weather in Tokyo?"),
    Message.from_role_and_content(Role.ASSISTANT, "What's the weather in Tokyo?"),
]

In [ ]:
from openai_harmony import (
    Conversation,
    DeveloperContent,
    HarmonyEncodingName,
    Message,
    Role,
    SystemContent,
    load_harmony_encoding,
)

enc = load_harmony_encoding(HarmonyEncodingName.HARMONY_GPT_OSS)
convo = Conversation.from_messages(
    [
        Message.from_role_and_content(
            Role.SYSTEM,
            SystemContent.new(),
        ),
        Message.from_role_and_content(Role.DEVELOPER, DeveloperContent.new().with_instructions("Talk like a pirate!")),
        Message.from_role_and_content(Role.USER, "Arrr, how be you?"),
    ]
)
tokens = enc.render_conversation_for_completion(convo, Role.ASSISTANT)
print(tokens)
print("==========")
# Later, after the model responded …
parsed = enc.decode(tokens)
print(parsed)

In [ ]:
M_HARM_2 = [
    Message.from_role_and_content(
        Role.ASSISTANT,
        'User asks: "What is the weather in Tokyo?" We need to use lookup_weather tool.',
    ).with_channel("analysis"),
    Message.from_role_and_content(
        Role.ASSISTANT,
        "I will check that",
    ).with_channel("commentary"),
    Message.from_role_and_content(Role.ASSISTANT, '{"location": "Tokyo"}')
    .with_channel("commentary")
    .with_recipient("lookup_weather")
    .with_content_type("json"),
]
tokens = enc.render_conversation_for_completion(Conversation.from_messages(M_HARM_2), Role.ASSISTANT)
print(tokens)
print("==========")
# Later, after the model responded …
parsed = enc.decode(tokens)
print(parsed)